In [ ]:
# ⚠️ IMPORTANTE: Instale as dependências via terminal ANTES de executar este notebook!
#
# Você está usando Python 3.13, que pode ter problemas com TensorFlow 2.16+
#
# Execute no terminal (fora do notebook):
#
# cd "/Users/Rafael-Freitas/Desktop/Estudos/Python/IA/Minst mlp"
# pip install tensorflow numpy matplotlib scikit-learn jupyter
#
# OU use o arquivo requirements.txt:
# pip install -r requirements.txt
#
# Depois volte e execute a próxima célula!

print("⚠️ ATENÇÃO: Instale as dependências via terminal primeiro!")
print("Execute: pip install -r requirements.txt")
print("\nDepois execute a próxima célula para verificar.")


⏳ Instalando TensorFlow...


ERROR: Could not find a version that satisfies the requirement tensorflow>=2.16.0 (from versions: none)
ERROR: No matching distribution found for tensorflow>=2.16.0


CalledProcessError: Command '['/Users/Rafael-Freitas/Desktop/Estudos/venv/bin/python', '-m', 'pip', 'install', '-q', 'tensorflow>=2.16.0']' returned non-zero exit status 1.

In [1]:
# 🔬 Experimentos com MLP MNIST - Notebook Simplificado
# Para uso local ou no Google Colab

# Se estiver no Colab e precisar de versão específica do TensorFlow:
# !pip install -q tensorflow==2.16.0

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Importar funções auxiliares (certifique-se que utils.py está no mesmo diretório)
from utils import preprocess_mnist, add_gaussian_noise, plot_history

print('✓ TensorFlow version:', tf.__version__)
print('✓ NumPy version:', np.__version__)
print('✓ Imports concluídos com sucesso!')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 📁 Instruções de Uso:
#
# Google Colab:
#   1. Faça upload de utils.py e main.py (Files > Upload)
#   2. Execute: !ls -la *.py
#
# Local: Os imports da célula anterior já funcionam!

print("✓ Notebook pronto para uso")
print("ℹ️ Para análise completa, use: mnist_robustness_analysis.ipynb")

In [ ]:
# Opção 1: Executar main.py (gera modelos e outputs em diretório)
# !python main.py

# Opção 2: Executar experimento inline (código abaixo)
# Isso permite ver os resultados diretamente no notebook

print("Escolha uma opção:")
print("1. Para executar main.py, descomente a linha acima")
print("2. Para experimento inline, execute as próximas células")
print("\nRecomendação: Use o notebook 'mnist_robustness_analysis.ipynb' para análise completa!")

In [ ]:
# Esta célula carrega modelos já treinados e avalia
# Certifique-se de que os modelos existem no diretório 'outputs/'

import os

# Verificar se existem modelos salvos
model_dir = 'outputs'
if os.path.exists(model_dir):
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.h5')]
    print(f"✓ Encontrados {len(model_files)} modelos em '{model_dir}/'")

    # Carregar dados de teste
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = preprocess_mnist(x_train, x_test)
    x_test_noisy = add_gaussian_noise(x_test, std=0.2, seed=42)

    # Avaliar cada modelo encontrado
    for model_file in sorted(model_files):
        model_path = os.path.join(model_dir, model_file)
        print(f"\n{'='*60}")
        print(f"Avaliando: {model_file}")
        print('='*60)

        try:
            model = load_model(model_path)

            # Avaliar em dados limpos
            clean_loss, clean_acc = model.evaluate(x_test, y_test, verbose=0)
            print(f"✓ Dados limpos:   Loss={clean_loss:.4f}, Accuracy={clean_acc:.4f}")

            # Avaliar em dados ruidosos
            noisy_loss, noisy_acc = model.evaluate(x_test_noisy, y_test, verbose=0)
            print(f"✓ Dados ruidosos: Loss={noisy_loss:.4f}, Accuracy={noisy_acc:.4f}")

            # Calcular degradação
            degradation = (clean_acc - noisy_acc) * 100
            print(f"✓ Degradação:     {degradation:.2f}%")

        except Exception as e:
            print(f"❌ Erro ao carregar modelo: {str(e)}")
else:
    print(f"⚠️ Diretório '{model_dir}/' não encontrado!")
    print("Execute 'main.py' primeiro para gerar os modelos:")
    print("  !python main.py")
    print("\nOu use o notebook 'mnist_robustness_analysis.ipynb' para análise completa.")

## 📊 Experimento Rápido Inline

Se você quiser executar um experimento rápido diretamente no notebook (sem precisar executar `main.py`), use as células abaixo:


In [ ]:
# Experimento rápido: Teste único com inicialização HeUniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.optimizers import Adam

print("Carregando dados MNIST...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = preprocess_mnist(x_train, x_test)
x_test_noisy = add_gaussian_noise(x_test, std=0.2, seed=42)

print("Construindo modelo...")
model = Sequential([
    Dense(64, activation='relu', input_shape=(784,), 
          kernel_initializer=HeUniform(seed=42)),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Treinando modelo (5 épocas)...")
history = model.fit(x_train, y_train,
                   batch_size=128,
                   epochs=5,
                   validation_split=0.1,
                   verbose=1)

# Avaliar
print("\nAvaliando modelo...")
clean_loss, clean_acc = model.evaluate(x_test, y_test, verbose=0)
noisy_loss, noisy_acc = model.evaluate(x_test_noisy, y_test, verbose=0)
degradation = (clean_acc - noisy_acc) * 100

print("\n" + "="*60)
print("RESULTADOS:")
print("="*60)
print(f"✓ Accuracy (dados limpos):   {clean_acc:.4f}")
print(f"✓ Accuracy (dados ruidosos): {noisy_acc:.4f}")
print(f"✓ Degradação:                {degradation:.2f}%")
print("="*60)


## 🎯 Recomendações

### Para análise completa e detalhada:
📓 **Use: `mnist_robustness_analysis.ipynb`**
- Compara 4 inicializadores diferentes
- Gera gráficos comparativos
- Tabelas de resultados
- Análise interpretativa

### Para experimentos programáticos:
🐍 **Use os scripts Python:**
- `quick_test.py` - Teste de instalação
- `load_and_run_config.py` - Configurações pré-definidas
- `run_custom_experiments.py` - Experimentos customizados
- `advanced_experiments.py` - Suite completa

### Documentação:
📚 Consulte:
- `QUICKSTART.md` - Início rápido
- `SIMULATION_SUGGESTIONS.md` - Ideias de simulações
- `EXPERIMENTS_GUIDE.md` - Guia completo
